In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile
import os

# Define the path to your file
file_path = '/content/drive/MyDrive/Data/archive.zip'

# Check if the file is a valid zip file
if not zipfile.is_zipfile(file_path):
    print("The provided file is not a valid zip file.")
else:
    # Define the directory to extract the contents to
    extracted_folder = '/content/dataset'

    # Create the target directory if it doesn't exist
    os.makedirs(extracted_folder, exist_ok=True)

    # Extract the contents of the zip file
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_folder)

    # List the extracted contents
    extracted_files = os.listdir(extracted_folder)
    print("Extracted files:", extracted_files)


Extracted files: ['Dataset']


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data generators
train_dir = '/content/dataset/Dataset/Train'
validation_dir = '/content/dataset/Dataset/Validation'
test_dir = '/content/dataset/Dataset/Test'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=4,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)


Found 140002 images belonging to 2 classes.
Found 39428 images belonging to 2 classes.
Epoch 1/4
4375/4375 [==============================] - 1924s 440ms/step - loss: 0.6263 - accuracy: 0.6444 - val_loss: 0.5540 - val_accuracy: 0.7309
Epoch 2/4
4375/4375 [==============================] - 1926s 440ms/step - loss: 0.4893 - accuracy: 0.7674 - val_loss: 0.4360 - val_accuracy: 0.8012
Epoch 3/4
4375/4375 [==============================] - 1890s 432ms/step - loss: 0.3876 - accuracy: 0.8303 - val_loss: 0.3932 - val_accuracy: 0.8290
Epoch 4/4
4375/4375 [==============================] - 1914s 437ms/step - loss: 0.3431 - accuracy: 0.8546 - val_loss: 0.4263 - val_accuracy: 0.8175


In [ ]:
# Save the model
model.save('/content/deepfake_detection_model.keras')

In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('/content/deepfake_detection_model.h5')

# Define the test data generator
test_datagen = ImageDataGenerator(rescale=1./255)

# Define the path to your test set
test_dir = '/content/dataset/Dataset/Test'

# Create the test generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_generator)
print("Test Accuracy:", accuracy)


Found 10905 images belonging to 2 classes.
341/341 [==============================] - 23s 66ms/step - loss: 0.4921 - accuracy: 0.7688
Test Accuracy: 0.768821656703949


In [3]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('/content/deepfake_detection_model.h5')

In [8]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Path to your test image
image_path = '/content/IMG20211029180359.jpg'
# Load the image, resize it to 224x224 (same as training), and convert to array
img = image.load_img(image_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions for batch

# Normalize the image
img_array = img_array / 255.0

In [9]:
# Predict
prediction = model.predict(img_array)

# If you have binary classification (real or fake)
if prediction[0] < 0.5:
    print("Prediction: Real")
else:
    print("Prediction: Fake")

1/1 [==============================] - 0s 82ms/step
Prediction: Real
